### Generative Adversarial Model - Group 9 <div style="text-align: right; float: right"> IANNWTF - Sheet08 </div>
In this sheet we used a generative model on the MNIST data set to generate "fake" handwritten numbers.

The training process is saved as summaries and saved in the `summary_dir` directory (it's called summary...). In tensorboard you can find the graphs for the losses, and can look at the images over the course of training. So please execute code to access graphs.

##### The usual imports

In [15]:
import numpy as np
import struct, os, time
import tensorflow as tf

##### Lukas' helper script for loading the mnist data

In [47]:
# %load recipes.py
import pymongo, json, numpy as np
from collections import Counter
from nltk.tokenize import RegexpTokenizer


class Recipes:

  def __init__(
      self,
      useMongo=True,
      config="config.json"
    ):

    with open(config) as cfg:
      self.data = json.load(cfg)
      storage = self.data['storage']
      CRAWL_FOLDER = storage['folder']
      RECIPE_COLLECTION = storage['collection']

    client = pymongo.MongoClient('mongodb://localhost:27017')
    self.db = client['iannwtf']
    self.coll = self.db[RECIPE_COLLECTION]

  def create_dictionaries(self):
    pass

  def ing2ids(self):
    pass
  
  def unit2ids(self):
    pass

  def words2ids(self):
    pass

  def ids2ing(self):
    pass
  
  def ids2unit(self):
    pass

  def ids2words(self):
    pass

  def get_ingredient_batch(self, batchsize=25, max_ingredients=15):
    count = 0
    batch = np.zeros((batchsize, max_ingredients), dtype=np.float)
    for doc in self.coll.find({},
                              {"ingredients.foodId":1, "_id":0},
                              no_cursor_timeout=True
                              ):
      _ids = [int(x['foodId']) for x in doc['ingredients']]
      if len(_ids) > max_ingredients:
          _ids = _ids[:max_ingredients]
      batch[count][:len(_ids)] = _ids[:]
      count += 1

      if count == batchsize:
          yield batch
          count = 0
          batch = np.zeros((batchsize, max_ingredients), dtype=np.float)


##### Lukas' layer helper file
The `back_conv_layer` was implemented by us. It is mostly analogue to the `conv_layer` function, with the biggest difference being the `target_shape` parameter, which holds the output shape to be genereated by the transposed convolution.

In [33]:
# %load 08_mnist-gan-layers.py
def feed_forward_layer(x, target_size, normalize = False, activation_function = None):
    print("Forward-Layer:" + str(x.shape))
    
    fan_in = int(x.shape[-1])
    
    if activation_function == tf.nn.relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [target_size], tf.float32, var_init)
    
    activation = tf.matmul(x, weights) + biases
    
    if normalize:
        activation = batch_norm(activation, [0])
    
    return activation_function(activation) if callable(activation_function) else activation


def conv_layer(x, kernel_quantity, kernel_size, stride_size, normalize = False, activation_function = False):
    print("Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]
    fan_in = int(x.shape[1] * x.shape[2])
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, depth, kernel_quantity], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    activation = tf.nn.conv2d(x, kernels, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation


def back_conv_layer(x, target_shape, kernel_size, stride_size, normalize = False, activation_function = False):
    print("De-Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]                   #number of activation maps in the de_conv layer (in_channels)
    fan_in = int(x.shape[1] * x.shape[2]) #resolution of the activation map, important for proper intitialization of kernel weights
    kernel_quantity = target_shape[-1]    #number of activation maps in the target layer (out_channel)
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
        
    # switch kernel_quantity and depth (in_channels and out_channels) because we do deconvolution
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, kernel_quantity, depth], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    # use conv2d_transpose as instructed
    activation = tf.nn.conv2d_transpose(x, kernels, target_shape, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation

def flatten(x):
    size = int(np.prod(x.shape[1:]))
    return tf.reshape(x, [-1, size])

def _pop_batch_norm(x, pop_mean, pop_var, offset, scale):
    return tf.nn.batch_normalization(x, pop_mean, pop_var, offset, scale, 1e-6)

def _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale):
    decay = 0.99
    
    dependency_1 = tf.assign(pop_mean, pop_mean * decay + mean * (1 - decay))
    dependency_2 = tf.assign(pop_var, pop_var * decay + var * (1 - decay))

    with tf.control_dependencies([dependency_1, dependency_2]):
        return tf.nn.batch_normalization(x, mean, var, offset, scale, 1e-6)

def batch_norm(x, axes):
    depth = x.shape[-1]
    mean, var = tf.nn.moments(x, axes = axes)
    global is_training
    
    var_init = tf.constant_initializer(0.0)
    offset = tf.get_variable("offset", [depth], tf.float32, var_init)
    var_init = tf.constant_initializer(1.0)
    scale = tf.get_variable("scale", [depth], tf.float32, var_init)
    
    pop_mean = tf.get_variable("pop_mean", [depth], initializer = tf.zeros_initializer(), trainable = False)
    pop_var = tf.get_variable("pop_var", [depth], initializer = tf.ones_initializer(), trainable = False)
    
    return tf.cond(
        is_training,
        lambda: _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale),
        lambda: _pop_batch_norm(x, pop_mean, pop_var, offset, scale)
    )


##### Loading the MNIST data and hyperparameters
`batch_size` is used for the fake and the real images

In [39]:
recipes = Recipes()

epochs = 2
learning_rate = 0.0004
beta1 = 0.5

#gen_feature_maps = [64, 32, 16] # target shapes for generator layers

# for tensorboard
weight_dir = "weights/"
summary_dir = "summary/"

## Hyperparameters
embedding_size = 64
lstm_memory_size = 64
ingredient_amount = 20000 # however many there are
cutoff_length = 300
subsequence_length = 300
batch_size = 50
epochs = 6
learning_rate = 0.05
dropout_rate = 0.85
optimizer = tf.train.AdamOptimizer
layer1_size = 512
layer2_size = 64
layer_out_size = 1

z_size = 50 # length of random input vector
max_ingredients = 15

starter_learning_rate = 0.05

#dis_feature_maps = [ 8, 16, 32] # target shapes for discriminator layers. maybe to be changed? dunno

In [40]:
def get_ings(embeddings, ingredients):
    '''
    embeddings: the weight matrix with all ingredients representations shape: [ingredient_amount, lstm_memory_size]
    ingredients: a matrix of ingedient vectors of who we want to find the nearest neigbhours
      
    '''''
    normed_embedding = tf.nn.l2_normalize(embeddings, dim=1)
    if (type(ingredients)!= list):
        arrays = ingredients
    #else: # we shouldn't have IDs, delete later
     #   ingredients = np.array(ingredients)
      #  arrays = tf.nn.embedding_lookup(embeddings, ingredients)
    
    # initalize "recipes". not sure yet how or if the loop thing is even the optimal way to do it
    
    recipes = []
    
    arraylist = tf.unstack(arrays)
    
    for array in arraylist : # get each recipe (sample in batch_size)
        
        normed_array = tf.nn.l2_normalize(array, dim=1) # what is this for?
  
        cosine_similarity = tf.matmul(normed_array, tf.transpose(normed_embedding, [1, 0]))
        
        closest_ings = tf.argmax(cosine_similarity, 1)  # shape [max_ingredients], type int64. we only need 1 ingredient, so no need for top_k anymore
        
        # stack this on top of recipes so we get batch_size, max_ingredients as output size in the end.
        # todo: actually implement.
        recipes = recipes + [closest_ings]
    
    recipes = tf.stack(recipes)
    return recipes # returns indices of closest ingredients. let's hope the dimension stuff works out in this function

In [62]:
# Defining the graph
tf.reset_default_graph()
   
###### The generator network. ######      
with tf.variable_scope("generator"):
  
    # see homework for original gen network

    ## placeholders
    with tf.variable_scope("inputs"):
        input_vec = tf.placeholder(tf.float32,[batch_size,z_size],name="input_vec") # for random input
        is_training = tf.placeholder(tf.bool, [],name="is_training")
        
        # what do the dimensions of this denote again?
        # i put *max_ingredients so it doesn't crash. like where I created the BasicLSTMCell a couple of lines later. how many "units" does it need?
        cell_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="cell_state")
        hidden_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="hidden_state")
        
        # original recipe placeholder. should be batch_size, max_ingredients, 3 later but for now we'll only take ingreidnets
        #orig_recipes = batch_norm(tf.placeholder(tf.int64, [batch_size,max_ingredients]), [0,1,2])

        orig_recipes_placeholder = tf.placeholder(tf.float32, [batch_size,max_ingredients], name="orig_recipes")
        orig_recipes = batch_norm(orig_recipes_placeholder, [0,1])

    ##### rnn
    with tf.variable_scope("rnn"):
        cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_memory_size*max_ingredients)
        cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_rate)
        zero_state = cell.zero_state(batch_size, tf.float32)
        state = tf.nn.rnn_cell.LSTMStateTuple(c = cell_state, h = hidden_state) # should be fine so far? no changes needed?
        
        outputs, state = tf.nn.static_rnn(cell, [input_vec], initial_state = state)
        
        # so at least in the example, lstm_memory_size == embedding_size.
        # so what do we do about the subsequence length?
        # for now I'll just ignore since we haven't even used sequences but the input_vec only. veeeery unsure about
        # this though.
        # so here I will state what the output should look like but not sure how we get the LSTM to produce this exact amount of data
        outputs = tf.reshape(tf.concat(outputs, axis=1), [batch_size, max_ingredients, lstm_memory_size])


    ##### embedding
    with tf.variable_scope("embedding"):
        init = tf.random_uniform_initializer(-1.0,1.0)
        embeddings = tf.get_variable("embedding", [ingredient_amount, embedding_size], initializer=init)

        ##fake_recipes will have dimension batch_size*max_ingredients*3 just like the original ones (for now without 3)             
        # so get_ings will have to return batch_size*max_ingredients for now.
        # might or might not work at the current state :D
        fake_recipes = get_ings(embeddings,outputs) 
                
    train_gen = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)


# create input batch for discriminator
batch = tf.concat([tf.cast(fake_recipes,dtype = tf.float32), orig_recipes],axis=0)

##### The discriminator network #####
# how do I adapt this?
train_dis = []
with tf.variable_scope("discriminator"):
    with tf.variable_scope("layer1"):
        l1 = feed_forward_layer(batch,64,False,None)
    with tf.variable_scope("readout"):
        logits = feed_forward_layer(l1,1,False,None)
    
    # get trainable variables for discriminator
    train_dis = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

##### Evaluation and training of the network #####
with tf.variable_scope("evaluation"):
    
    # to compute cross entropy for generator. consists of only "1" because this is the discriminator label for "real"
    gen_labels = tf.ones((batch_size,1))
    
    # only the first half of the output from the discriminator concerns the pictures produced by the generator
    # so only get first half of logits (which equals the batch_size)
    gen_logits = logits[:batch_size]
    
    # for discriminator cross entropy. first half of input are fake images ("0"), second half real ones ("1")
    dis_labels = tf.concat((tf.zeros((batch_size,1)),tf.ones((batch_size,1))),axis=0)
    dis_logits = logits
    
    gen_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=gen_labels, logits=gen_logits)
    dis_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=dis_labels, logits=dis_logits)
    
    # tensorboard stuff
    tf.summary.scalar("generator_loss", tf.reduce_mean(gen_loss))
    tf.summary.scalar("discriminator_loss", tf.reduce_mean(dis_loss))
    
    global_step = tf.get_variable("global_step",[],tf.int32,trainable=False)
    
    # initialize optimizer
    optimizer  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
    
    # define training steps with respective variable lists
    gen_step = optimizer.minimize(gen_loss, var_list=train_gen, global_step=global_step)
    dis_step = optimizer.minimize(dis_loss, var_list=train_dis, global_step=global_step)
    
    summaries = tf.summary.merge_all()

Forward-Layer:(100, 15)
Forward-Layer:(100, 64)


##### This is where we train our network

In [ ]:
# HAVE ONLY MADE SMALL ADJUSTMENTS. don't think there's a point if we don't have data


with tf.Session() as session:
    ## initialising directories if they do not exist
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)
    train_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(summary_dir, session.graph)
    
    ## Try to load trained data and initialise variables if there is none
    ckpt = tf.train.latest_checkpoint(weight_dir)
    if ckpt is None:
        print("No checkpoint found. Initialising variables.")
        session.run(tf.global_variables_initializer())
    else:
        train_saver.restore(session, ckpt)
    
    ## Train all the epochs and save the summaries for each timestep
    for epoch in range(1, epochs+1):
        
        print("Starting epoch %d..." %epoch, end='')
        
        real_data = recipes.get_ingredient_batch(batch_size, max_ingredients) # get batch of original recipes #mnist.get_batch(batch_size)
        t = time.time()
        
        for data in real_data:
            
            _state = session.run(zero_state)
            
            ## for each batch we need a new set of random vectors
            z_vec = np.random.uniform(-1, 1, (batch_size,z_size))
            
            # feed data into placeholders
            _feed_dict = {input_vec: z_vec, 
                         orig_recipes_placeholder: data,
                         is_training: True,
                         cell_state: _state.c,
                         hidden_state: _state.h,}
            
            # let's train our network!
            _gStep, _dStep, _summ, _step, _fr = session.run([gen_step, dis_step, summaries, global_step, fake_recipes],
                                                        feed_dict=_feed_dict)
            train_writer.add_summary(_summ, _step)
            
            print(_fr[0])
            
        t = time.time() - t
        est = t * (epochs - epoch)
        
        ## This is just the time formatting. Move along.
        m, s = divmod(t, 60)
        e_m, e_s = divmod(est, 60)
        
        print(("Finished epoch in {0: .0f}m{1: .2f}s. "+\
              "Estimated remaining time: {2: .0f}m{3: .2f}s").format(m,s,e_m,e_s))
        
        train_saver.save(session, weight_dir, global_step=_step)

No checkpoint found. Initialising variables.
Starting epoch 1...[15954 18597 17419 19301  7084 19130  3124 16416  4228   663  9911  4103
 16163 13600 13564]
[10286  4793 19039  5435  1624 19458 19841 15732  5472 14215  6162 14440
  7572 10740  9734]
[18363 17949  6060 17318 18237  6901  3261 13527  9951 13276    24 18305
  6435 19597 18704]
[ 4368  8083 17596 16634  1592 12905  5839   196 15704  4149  5630 10307
 11906 17793  7543]
[ 9742 13189  5697  2506   279   674  5127  6148  7280 12092 11870  8261
   467 16712  9557]
[ 5410 10576  5056 10648  9686 19023  2236 15377  9852 10608 16823 15343
 18011 12254  1062]
[14090 17194  7072  9880 11723 17981 12553 16003 17542  5598 14901 10508
  4176 18482 16499]
[ 3494  1734 13871 14126 17980  8333 14304  6366  9080 14395  8410  3507
  4374  2015 16712]
[ 5203 13587 15360 19119  7018  5535  8477  9029  2802 16384 18314   875
 19965  6292  6939]
[ 7024 12538  2825  5889  9661  8871 11779  5290  3421  9302  7673 18789
 15695 15875 18798]
[11364

[19950  8144 13916  1235 17328 18774 16841 13072 15290  8786 10401  1209
 10581 14553 12431]
[14521 18599   843   655  3419  3472  2762 16844 17784  3272  7041 10196
 19039 19447  1770]
[ 6347 11940 13182  3110   147  3928 18359  5351  4838  8211 12366 12499
  9110  5076  8973]
[ 1575 14303 11493  5500 18354  6377 15731  5992 16569  6698 17600 15503
  7714  3623  8929]
[13836  3769 13645   741  7660 18085 15507 12859 16271 12352 14489 15435
   218  4932 14866]
[ 1944  1967 16211   355   365  5041  1086  5262 12363  4678  2153  9503
  1488  8646  6311]
[11989 19715 17302  9348  5697 15330  1402  1251   732  7616 18922  7735
 13184 11829 10697]
[ 1048 12657  7622  3773 11623 11008 11445  3202   162  8169  1218  5940
  7141 15928  7794]
[15304 16223  5475 11878   512 18196   653  1496  8467   176 19732 12674
  6932 13571 15917]
[ 5115  3412  4998 17449  6345  2810  8160 19140 16501 14802 16511  1969
 10422  4245  7905]
[10843 10718  2825   250  7105  3282  3748  1543  2795 16441 11546 118

[ 1284 10791  3703  5645 13674 11104 19233 11256 15627 14618   332 17643
  8043  8140 10176]
[ 7780  4101  9191   528  3085   384 17622  2449 16069  7341  7752 11961
 16656  1849 17149]
[15022  6768  9971 10849  7776 11568 16895 18911   796  1611 17398 19476
  8385  4224 15256]
[18269 11394 13923   695 11751  6475 11429 12200 13034   570 12415  6325
  3598 18496 14315]
[19670  5195  1580  9152  9739 14971 18049 10981  8846 14003 18780 12353
 12773 18227 13137]
[10120  1600  9580  2708  4840  6828  8904 12374 12635  5806  4412 19183
  8130   629  6441]
[  316  1573  2326  6100 15633 13845 16380 16381  2562  1672  6832 12458
 15891 16428   234]
[ 6370  1528  2830 19040 18465 15770  3197 19050  8541  4895   803 11999
  9020 10372  9621]
[15800  3103  7684 17017  2672 18195 13450  9479  1862  7402  2629  6694
 18465  8886 16380]
[18065 13467   301 15020  7512 17608  3713  8794  7691  9213 17512  9021
  9793 12927 11652]
[ 5125 14761 14717  9418  9015  3615 12901  4682 17665 17060  7402  87

[18311  5943 13283 15315   977 16990 11104 18263  3909  9686  5984 10351
 18153  3402  9170]
[ 3981  7638   843 18353 10213 10299  6712  8038  6588  4980 10543  4489
  5983  4189 18566]
[ 9361 12811  4455  3626  6369  4414 17330 14604 18855  7280  3467 11870
  2931 15267  1039]
[ 2935   180 16438  1736  8651 16138 10991     9 12574  1549  3517 17921
 13146 19178 11432]
[ 3249 16594 11814  4774   776  2576 16810 18617  8635  8798  9728 17359
 13437   343  8522]
[ 7866 15775 12294 17426 15587 16875 10354  2284 16284 11978  6031  4446
  4807  2388  2530]
[ 6247 14875 16968 16388  2855 15556 15842 12232 14120 16920 12892  5729
 19310  1245  3436]
[ 5957  6449 11553  8804  4054  5471 12345 17389  2360  4142 12591  5807
   388 16720 17230]
[15162 16927 12290 19459  4708  3748  6489 18100  7307  4466 12598  1702
  6910  8723  3824]
[ 7097  2973 11671  7551  6626  9246 16108 14547  7442  1781  3305 15414
 17167 13413 13176]
[12564 15218 10712  9719 12388  3758   492  7146  2802  5304  5998 191

[ 1317  4251 17189  9461  1842 15891  7926 10292 14156 10607  5689 17181
 17804  4767 19902]
[ 2556  2483  2207 19768 16604  8688  5956  6835  3140   824 13294 10917
 17347  3845  6219]
[ 2528  5875 11657  1783 18758  6236  1447  3833  4727   277   541  5828
  9978  6731  6652]
[14438  7288 10875 13595   771 16088  7199    28  1006  5435  7308 11700
 13702  5415 17908]
[ 3411  4857 19046 18268  6766  5202  6187 15797  7300  8669  8533 10745
 14947  5386  1235]
[10522  1301  4881 16642  5617  7910  4056 14907 17234 15246 17012 18098
 18666  5476 19499]
[ 4862 16891  9122    38 13653  2525  2948  1661  5111 16071  8215 11023
   850 14881  6266]
[ 6563 15259  4134  7090   294 17519   739  5069 17504 19195  7166  3910
 14823 16436  4069]
[13626 19789 15052 11420  9710  7712  4671 18354 17300  6708 19326 13761
 14943 19688 18358]
[ 8821  4472  9633  1014 16634 17314  6410 13974 10067  5540 10323  7057
 11530 14993  7873]
[ 4995  4684  4887  8597  8835 14856  2701  1810   476   625 19694  57

...

[10877 10762   131 10954  9648 10272 17209   961 14235 18283 17328   396
   218  1606  5316]
[18649  2864  9362   369  6825 10885  8177 12904 17412  5075  4968 16606
 10665 14156 18046]
[15492 17105  5991 18380  6903  3598 16629 11354 13293 10339 16799 14515
 15502  4421  8361]
[ 7809 15632  1800 15703 16983 14587 16559  7820 10315  6136 15569  4797
 18303 12902  7529]
[ 3869  7796 14997 17102  5283  9205 17933  8669  8399 10239  8972 18993
 19527  3959 16121]
[ 5758 12805   374  5858  9133 12347 18751 10292 13464  3588 12711  3587
  4941 12955  2042]
[18439 13578  9187 12620 12987  1720 14439 13964 12035  3202  2466 10165
    99  3200 17833]
[19189  9646  5888  9979 12136 10800 10488   745 15355  2961 17167  9301
  1308  2959 17607]
[18762  1535 11653 11385  5830 17334 18607  2900  9825 13922 18112 10061
  3589 19950 18022]
[  816 10780  5366  1774  5286  9656 17581  6828 11723 19772 11429  6139
 15195 12455  3371]
[ 7148 10139  9251  8102  4321  6062  9438  2130 13939  7280 11287 111

[19416  1207 19088 13573   539  8943  4105 16024  9502 15935  8003  8520
  3876 17823 10121]
[10442 19326  9728  7102  4995 13330  6818   513  5504 10220  2529  6585
  7782 15812 10473]
[14160 18575 17885  5971 13305 12755  8503    77 17669  5967  8428 18365
  5602 16114 11252]
[11118 14674  3744   649 15574 18773  2283  7239 17460  5850  7683 12519
  7648   244  6616]
[12893     1  9439 18357  5035 16061  2643 12995   640  2064  6581  4058
  1482  9088  3747]
[  154 18012  8089  2216 17362 13964 14866 19913 15877   556 13707  5686
  8778  1595 19405]
[ 1001 15135 19350 10725  4121 12820 19734 18786  9386 13428  9273 16041
  1406  6497  4151]
[ 3644 14271   701  9050  4328  5576 17561  2937   117  1757  8634 14417
 17717 17036 18068]
[18548  1678  7304  8954  1650  1791 14130  4273  5434 17030 15516 17108
 13459  2187  3566]
[17063  2761  1261  9772 17760 19917 13201 18865 11113 18639 14415 15359
  6325  3927  8851]
[10048 11328  2532 12043  2430 13353 18193 18162  4295 19746 17107 139

[19019 10584 12931 16366 14789  1208 12838  7481 10514 16824 13170  7679
 10781  8192  5317]
[ 3644   508   560  1104 13918 13001  8124   200  6212 18265  2629 19356
 15727   954  7891]
[13857 19822  7353  3956 16441  8016  7102 10339  1160 16200  4534 13584
 19189 10620 16854]
[15004 10064 14647 10259  5585 19975 16407  1712  9604 18008 12558  3925
 19165  1992  7489]
[ 6086 11978  2478 14755   271  8667 12254 16366 13020  1043 15637 15800
 16471   700 12772]
[12772 17955  9744 15302   369 12849  7575 10793  9379 13543  4959 12747
 15587  3651 19027]
[12971  3393 11493 18714  3490  2587 12755 19300  9814 17967  5016  3346
  6597 17361  2040]
[ 8296  7362  3672 17207 17431  1332 12617 19096 14814 10185 13719 11022
 12164  6376 19872]
[ 6129   993 18310 18524  8855  2518  5781 14818 13026 13734 10847 17356
 15453  4608  4213]
[ 1514  9047  6625   185  9146  3670 17866  5536 15195   872 11715 11497
 15049  9980 16856]
[10040  6744  3333 17997 14297  7811 18521 12535  4478 15228 18036 103

[ 4101  3193  5039 17524 11898 18156 14364 11108 18268 13403 18123 14119
  6077 19032 12824]
[ 9971 18161  9971  9878 19738  2030  3263 15555  7315  3826 19738  7254
 14143 11615 11046]
[10796  3859 10106 18422  4004  9239 18406 13409 10116  5873 13730  8613
  9309 19646  2315]
[ 5973 19861  3954  7035  5922  2886 11993 18722 14948  2796  4102  6692
 14519 18591   939]
[12357 15560  9335  2489 16485 10935 18208  9900 16174  3705 15531 11105
  6456 12355 18998]
[ 5274 12297 12256 14790 19518 10802 14384 11543 11988   328 14130 12277
 15250 10621    64]
[ 9771 11231 17097 13224 11954  9217 11923 14436 18251  5572 11555 13731
 14661   871 12262]
[13140  6978 17808  5205 11783  5061 12539 18550  2593 17820 19824 15197
 15708  1210 14864]
[ 3361  6206 17820  2484  4913 16240  6691 13970   921 14450 19057  2126
  1366  6915  4984]
[ 2608  3572  1296 18422 17477 16534 16206 12774 15147 10863  4987   939
  6638 13420 16049]
[14977 18834  9062 15125  5423 17325  9813   761  5642  5551 10428 139

[ 9516  5014  1346  7677   501  8737 18411 12936  6967 11295 15926   687
 14377 16322  9754]
[ 4429 11508 14169 17114 13305 11041  7755 18229 12779  3119  8762  4764
  7908   763 14353]
[ 9421 16615  6051 13144  7177   615  3346   315  6697 12715  2666 16392
  4416 17404  2687]
[11011  6062 18789 15418  2159  9186  2187  4779 11145 17996  3133   359
 15665 15381 14946]
[10419 16994  7398  7852  1760 19976 13730  6587  1451 17181  9889  4389
  7356 14094  8114]
[12823 13664  9745  5031  7507 10706  8377  3545 15518 19942  4141 17789
 10983 11964  6637]
[14351 12426  2895 15083 15275 15571 14979  7123 11465 10209 10981  7255
  4134 10970  1765]
[11273  5255  5986 12084 11747  8069  8500 10929  9659  5273 13077 15599
 17668 12440  7812]
[11576 14402  6791 14346  5204 11454   718  3341  4957  8085  2092 13162
 12079 16681  6612]
[12833 12623 19675  9024 10931  4875  8559 17550 19695  3336 19850 19530
 14702 18130  5754]
[ 5434  8593  7597 13143  9445  6330   982 19285 10873  1994 16701 196